In [1]:
import pandas as pd
import numpy as np
import google.generativeai as genai
import os
from sklearn.metrics import mean_absolute_error
import time

# configure rating model
genai.configure(api_key=os.environ["API_KEY"])

orig_chat_file = pd.read_csv("chats.csv")
MAX_TOKENS = 1
TEMP = 0

/Users/selinameyer/Documents/GitHub/personal-assistant-for-communication-analysis-and-improvement/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [174]:
def get_interaction_context_text(row):
    # Mapping interaction contexts to corresponding few-shot text
    few_shot_prompt = ""
    if row["interaction_context"] == "Relationship":
        few_shot_prompt += "Interaction Context: Whatsapp conversation between a romantic couple: \n"
    elif row["interaction_context"] == "friendship":
        few_shot_prompt += "Interaction Context: Whatsapp conversation between friends: \n"
    elif row["interaction_context"] == "workplace-no-hierarchy":
        few_shot_prompt += "Interaction Context: Email conversation between colleagues: \n"
    elif row["interaction_context"] == "Workplace-hierarchy":
        few_shot_prompt += "Interaction Context: Email conversation between employees and their boss: \n"
    return few_shot_prompt

def get_conversational_context_text(row):
    # Mapping advice scenarios to corresponding few-shot text
    few_shot_prompt = ""
    if row["context"] == "Advice for victim":
        few_shot_prompt += f"""Conversation Context: Advice for {row["Victim"]} who felt uneasy after the conversation\n"""
    elif row["context"] == "Advice for culprit":
        few_shot_prompt += f"""Conversation Context: Advice for {row["culprit"]} who felt like they might have done something wrong after the conversation\n"""
    else:
       few_shot_prompt +=  f"""Conversation Context: Advice for {row["Victim"]} who didn't know how to react next.\n"""
    return few_shot_prompt

## Numerical rating

In [193]:
# read in training and test set
train = pd.read_csv("advice_quality_train_rating.csv", index_col="Unnamed: 0")
test = pd.read_csv("advice_quality_test_rating.csv", index_col="Unnamed: 0")

In [201]:
# Create Few-Shot Prompt
few_shot_prompt = "Judge if the advice given is of high quality given the interaction context and relationship between the speakrers. \n"
for index, row in test.iterrows():
    few_shot_prompt += get_interaction_context_text(row)
    few_shot_prompt += f"""Chat: ```{row["chat_text"]}```\n"""
    few_shot_prompt += get_conversational_context_text(row)
    few_shot_prompt += f"""Advice: {row["advice_text"]}\n"""
    few_shot_prompt += "On a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.\n"
    few_shot_prompt += f"Explanation: {row["text_feedback"]}\n Rating: {row["rating"]}\n"


In [202]:
# Configure model with system instruction
model = genai.GenerativeModel('gemini-1.5-flash',
                             system_instruction="""You are a communication expert tasked with judging the quality of advice given for different interactions between people.
                             Explain your Rating first, then return your rating of the most recent advice on a scale of 1 (very bad) to 5 (very helpful)""")

In [203]:
# Solicit numerical advice ratings from Gemini
responses = {}

for index, row in train.iterrows():
    responses[index] = {}
    responses[index]["advice_id"] = row["advice_id"]
    responses[index]["rating"] = row["rating"]

    this_few_shot_prompt = few_shot_prompt

    this_few_shot_prompt += get_interaction_context_text(row)
    this_few_shot_prompt += f"""Chat: ```{row["chat_text"]}```\n"""
    this_few_shot_prompt += get_conversational_context_text(row)
    this_few_shot_prompt += f"""Advice: {row["advice_text"]}\n"""
    this_few_shot_prompt += "On a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.\n"
 
    try:
        response = model.generate_content([(few_shot_prompt)],
                                          request_options={"timeout": 1000})
        responses[index]["model_output"] = response.text
    except Exception as e:
        time.sleep(120)
        response = model.generate_content([(few_shot_prompt)],
                                          request_options={"timeout": 1000})
        responses[index]["model_output"] = response.text
    
    time.sleep(4)
    df = pd.DataFrame.from_dict(responses, orient='index')
    df.to_csv("Gemini-advice-judgements_numerical_ratings.csv")

In [204]:
# Extracting the model's numerical ratings and calculating mean absolute error
df['model_rating'] = df['model_output'].str.extract(r'Rating:\s+(\d+)', expand=False)
mean_absolute_error(df["rating"], df["model_rating"])